In [ ]:
import pickle
import numpy as np
from datetime import datetime
import random
import string

# Global variables
accounts_file = "accounts.pkl"
transactions_file = "transactions.pkl"

# Account class
class Account:
    def __init__(self, name, account_type, balance=0):
        self.name = name
        self.account_number = self.generate_account_number()
        self.account_type = account_type
        self.balance = balance
        self.transactions = []

    def generate_account_number(self):
        # Generate a random and unique 10-digit account number
        while True:
            account_number = ''.join(random.choices(string.digits, k=10))
            if account_number not in accounts:
                return account_number

    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            self.transactions.append(("Deposit", amount, datetime.now()))
            print(f"Deposited ₹{amount}. New balance: ₹{self.balance}")
        else:
            print("Invalid deposit amount.")

    def withdraw(self, amount):
        if amount > 0 and amount <= self.balance:
            self.balance -= amount
            self.transactions.append(("Withdrawal", amount, datetime.now()))
            print(f"Withdrew ₹{amount}. New balance: ₹{self.balance}")
        else:
            print("Invalid withdrawal amount or insufficient balance.")

    def transfer(self, target_account, amount):
        if amount > 0 and amount <= self.balance:
            self.balance -= amount
            target_account.balance += amount
            self.transactions.append(("Transfer Out", amount, datetime.now()))
            target_account.transactions.append(("Transfer In", amount, datetime.now()))
            print(f"Transferred ₹{amount} to Account {target_account.account_number}.")
        else:
            print("Invalid transfer amount or insufficient balance.")

    def get_transaction_history(self):
        return self.transactions

    def get_summary_statistics(self):
        amounts = [t[1] for t in self.transactions]
        if amounts:
            return {
                "Total Deposits": np.sum([t[1] for t in self.transactions if t[0] == "Deposit"]),
                "Total Withdrawals": np.sum([t[1] for t in self.transactions if t[0] == "Withdrawal"]),
                "Average Transaction": np.mean(amounts),
            }
        return {"Total Deposits": 0, "Total Withdrawals": 0, "Average Transaction": 0}

    def view_passbook(self):
        print("\nPassbook for Account:", self.account_number)
        print("Account Holder:", self.name)
        print("Account Type:", self.account_type)
        print("Current Balance: ₹", self.balance)
        print("\nTransaction History:")
        print("{:<20} {:<15} {:<10} {:<10}".format("Date", "Type", "Amount (₹)", "Balance"))
        balance = self.balance
        for transaction in reversed(self.transactions):
            if transaction[0] == "Deposit":
                balance -= transaction[1]
            elif transaction[0] == "Withdrawal":
                balance += transaction[1]
            elif transaction[0] == "Transfer Out":
                balance += transaction[1]
            elif transaction[0] == "Transfer In":
                balance -= transaction[1]
            print("{:<20} {:<15} {:<10} {:<10}".format(
                transaction[2].strftime("%Y-%m-%d %H:%M:%S"),
                transaction[0],
                transaction[1],
                balance
            ))

    def __str__(self):
        return (f"Account Holder: {self.name}\n"
                f"Account Number: {self.account_number}\n"
                f"Account Type: {self.account_type}\n"
                f"Current Balance: ₹{self.balance}\n")


# File handling functions
def load_data():
    try:
        with open(accounts_file, "rb") as f:
            accounts = pickle.load(f)
    except (FileNotFoundError, AttributeError):
        accounts = {}
    try:
        with open(transactions_file, "rb") as f:
            transactions = pickle.load(f)
    except (FileNotFoundError, AttributeError):
        transactions = {}
    return accounts, transactions


def save_data(accounts, transactions):
    with open(accounts_file, "wb") as f:
        pickle.dump(accounts, f)
    with open(transactions_file, "wb") as f:
        pickle.dump(transactions, f)


# Main program
def main():
    global accounts
    accounts, _ = load_data()

    while True:
        print("\nBank Account Management System")
        print("1. Open a new account")
        print("2. View account details")
        print("3. Perform transactions")
        print("4. View transaction history")
        print("5. View passbook")
        print("6. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Enter account holder's name: ")
            account_type = input("Enter account type (savings/current): ")
            initial_balance = float(input("Enter initial balance: "))
            account = Account(name, account_type, initial_balance)
            accounts[account.account_number] = account
            print(f"Account created successfully. Account Number: {account.account_number}")

        elif choice == "2":
            account_number = input("Enter account number: ")
            account = accounts.get(account_number)
            if account:
                print(account)
            else:
                print("Account not found.")

        elif choice == "3":
            account_number = input("Enter account number: ")
            account = accounts.get(account_number)
            if account:
                print("1. Deposit")
                print("2. Withdraw")
                print("3. Transfer")
                transaction_choice = input("Enter transaction type: ")
                if transaction_choice == "1":
                    amount = float(input("Enter deposit amount: "))
                    account.deposit(amount)
                elif transaction_choice == "2":
                    amount = float(input("Enter withdrawal amount: "))
                    account.withdraw(amount)
                elif transaction_choice == "3":
                    target_account_number = input("Enter target account number: ")
                    target_account = accounts.get(target_account_number)
                    if target_account:
                        amount = float(input("Enter transfer amount: "))
                        account.transfer(target_account, amount)
                    else:
                        print("Target account not found.")
                else:
                    print("Invalid choice.")
            else:
                print("Account not found.")

        elif choice == "4":
            account_number = input("Enter account number: ")
            account = accounts.get(account_number)
            if account:
                print("\nTransaction History:")
                for transaction in account.get_transaction_history():
                    print(f"{transaction[2]}: {transaction[0]} of ₹{transaction[1]}")
                print("\nSummary Statistics:")
                stats = account.get_summary_statistics()
                for key, value in stats.items():
                    print(f"{key}: ₹{value}")
            else:
                print("Account not found.")

        elif choice == "5":
            account_number = input("Enter account number: ")
            account = accounts.get(account_number)
            if account:
                account.view_passbook()
            else:
                print("Account not found.")

        elif choice == "6":
            save_data(accounts, {})
            print("Exiting program. Data saved.")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()


Bank Account Management System
1. Open a new account
2. View account details
3. Perform transactions
4. View transaction history
5. View passbook
6. Exit


Enter your choice:  1
Enter account holder's name:  Ayushi 
Enter account type (savings/current):  saving
Enter initial balance:  50000


Account created successfully. Account Number: 9536123082

Bank Account Management System
1. Open a new account
2. View account details
3. Perform transactions
4. View transaction history
5. View passbook
6. Exit


Enter your choice:  2
Enter account number:  9536123082


Account Holder: Ayushi 
Account Number: 9536123082
Account Type: saving
Current Balance: ₹50000.0


Bank Account Management System
1. Open a new account
2. View account details
3. Perform transactions
4. View transaction history
5. View passbook
6. Exit


...

1. Deposit
2. Withdraw
3. Transfer


Enter transaction type:  3
Enter target account number:  9536123082
Enter transfer amount:  5000


Transferred ₹5000.0 to Account 9536123082.

Bank Account Management System
1. Open a new account
2. View account details
3. Perform transactions
4. View transaction history
5. View passbook
6. Exit


Enter your choice:  5
Enter account number:  9536123082



Passbook for Account: 9536123082
Account Holder: Ayushi 
Account Type: saving
Current Balance: ₹ 53000.0

Transaction History:
Date                 Type            Amount (₹) Balance   
2025-02-01 21:18:50  Transfer In     5000.0     48000.0   
2025-02-01 21:18:03  Transfer Out    4000.0     52000.0   
2025-02-01 21:16:57  Transfer In     2000.0     50000.0   

Bank Account Management System
1. Open a new account
2. View account details
3. Perform transactions
4. View transaction history
5. View passbook
6. Exit
